## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
## Solution 22 - not optimal
## Solution 22 - not optimal

In [ ]:
for my_int = 1:144
    
    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=600))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min = jvalue(vars["f_min"])
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])

    marge_dies = jvalue(vars["marge_dies"])

    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Min frequency limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0

        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]

    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
        status    = string(status),
        interval  = interval,
        P_bat_set = getvalue(P_bat_set[:,:]), 
        SOC_bat   = getvalue(SOC_bat[:,:]),
        P_PV_set  = getvalue(P_PV_set[:,:]),
        P_PV      = getvalue(P_PV[:,:,:]),
        ON_dies   = getvalue(ON_dies[:,:]),
        P_dies    = getvalue(P_dies[:,:,:,:]),
        P_bat_cha = getvalue(P_bat_cha[:,:]),
        P_bat_dis = getvalue(P_bat_dis[:,:]),
        X_bat     = getvalue(X_bat[:,:]))
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/144")
    
    if string(status)=="Infeasible"
        break
    end
    
end

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]
Presolve removed 7288 rows and 8726 columns
Presolve time: 2.42s
Presolved: 385832 rows, 197482 columns, 1192061 nonzeros
Variable types: 192874 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2508950e+07   2.065022e+07   0.000000e+00     12s
   49251    3.6845817e+07   2.371590e+06   0.000000e+00     15s
   73404    3.6782522e+07   2.050721e+06   0.000000e+00     20s
   96445    3.6746236e+07   1.769507e+06   0.000000e+00     25s
  117403    3.6706319e+07   1.551280e+06   0.000000e+00     30s
  137188    3.6650473e+07   1.338576e+06   0.000000e+00     35s
  1563

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 22.264555686

Presolve removed 7282 rows and 8720 columns
Presolve time: 2.61s
Presolved: 385838 rows, 197488 columns, 1192080 nonzeros
Variable types: 192880 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2763538e+07   2.082871e+07   0.000000e+00     14s
   40084    3.7009337e+07   2.592378e+06   0.000000e+00     15s
   65272    3.6956640e+07   2.161978e+06   0.000000e+00     20s
   88731    3.6911727e+07   1.859535e+06   0.000000e+00     25s
  111158    3.6871853e+07  

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [3e-01, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 571.560290936

Presolve removed 7296 rows and 8734 columns
Presolve time: 2.26s
Presolved: 385824 rows, 197474 columns, 1192042 nonzeros
Variable types: 192866 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2567759e+07   2.062950e+07   0.000000e+00     12s
   46183    3.7198602e+07   2.491942e+06   0.000000e+00     15s
   71682    3.7138974e+07   2.078172e+06   0.000000e+00     20s
   95224    3.7107003e+07   1.790840e+06   0.000000e+00     25s
  117484    3.7063070e+07 

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 378.193043281

Presolve removed 7292 rows and 8731 columns
Presolve time: 2.25s
Presolved: 385828 rows, 197477 columns, 1192055 nonzeros
Variable types: 192869 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3068705e+07   2.080792e+07   0.000000e+00     12s
   47741    3.7324291e+07   5.490583e+06   0.000000e+00     15s
   71831    3.7265888e+07   2.184847e+06   0.000000e+00     20s
   93166    3.7227328e+07   1.851866e+06   0.000000e+00     25s
  117530    3.7180376e+07 

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [2e-01, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 1707.584688927

Presolve removed 7287 rows and 8725 columns
Presolve time: 2.23s
Presolved: 385833 rows, 197483 columns, 1192070 nonzeros
Variable types: 192875 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2424431e+07   2.063028e+07   0.000000e+00     12s
   47703    3.6705812e+07   2.448170e+06   0.000000e+00     15s
   71774    3.6645829e+07   2.129490e+06   0.000000e+00     20s
   95732    3.6601059e+07   1.792183e+06   0.000000e+00     25s
  117445    3.6556164e+07

...

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [3e-01, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 857.911745517

Presolve removed 7284 rows and 8722 columns
Presolve time: 2.40s
Presolved: 385836 rows, 197486 columns, 1192092 nonzeros
Variable types: 192878 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2773594e+07   2.020330e+07   0.000000e+00     13s
   43517    3.7152646e+07   2.578712e+06   0.000000e+00     15s
   68745    3.7064790e+07   2.150563e+06   0.000000e+00     20s
   92947    3.6974834e+07   1.860542e+06   0.000000e+00     25s
  117967    3.6874709e+07 

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 173.409441162

Presolve removed 7271 rows and 8709 columns
Presolve time: 2.28s
Presolved: 385849 rows, 197499 columns, 1192120 nonzeros
Variable types: 192891 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2505991e+07   2.014855e+07   0.000000e+00     13s
   44571    3.6805032e+07   2.504007e+06   0.000000e+00     15s
   69181    3.6731587e+07   2.237598e+06   0.000000e+00     20s
   95650    3.6640073e+07   1.851014e+06   0.000000e+00     25s
  120281    3.6564148e+07 

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 1123.688879934

Presolve removed 7294 rows and 8732 columns
Presolve time: 2.29s
Presolved: 385826 rows, 197476 columns, 1192051 nonzeros
Variable types: 192868 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2499470e+07   2.017638e+07   0.000000e+00     13s
   43472    3.6845221e+07   2.542151e+06   0.000000e+00     15s
   68364    3.6758049e+07   2.193047e+06   0.000000e+00     20s
   93123    3.6671533e+07   1.937697e+06   0.000000e+00     25s
  118328    3.6585876e+07

Academic license - for non-commercial use only
Optimize a model with 393120 rows, 206208 columns and 1215359 nonzeros
Variable types: 201600 continuous, 4608 integer (4608 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R345600 by 189.865557641

Presolve removed 7288 rows and 8726 columns
Presolve time: 2.26s
Presolved: 385832 rows, 197482 columns, 1192067 nonzeros
Variable types: 192874 continuous, 4608 integer (4608 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2616208e+07   2.030073e+07   0.000000e+00     13s
   43334    3.6920451e+07   2.546210e+06   0.000000e+00     15s
   69486    3.6824881e+07   2.190076e+06   0.000000e+00     20s
   93699    3.6751187e+07   1.868216e+06   0.000000e+00     25s
  118650    3.6678081e+07 

Excessive output truncated after 529763 bytes.